In [128]:
print(14)

14


In [129]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
df = pd.read_csv('ASML.csv',parse_dates = ["Date"],index_col = 0).dropna()
df_plot = df[{'Open','High','Low','Close'}]

In [130]:
df_plot.tail()

,Low,Close,High,Open
Date,,,,
2021-03-05,508.329987,527.039978,530.849976,527.969971
2021-03-08,501.109985,502.190002,529.169983,526.440002
2021-03-09,524.950012,539.520020,543.210022,527.640015
2021-03-10,521.109985,523.330017,541.820007,540.080017
2021-03-11,540.010010,544.400024,548.489990,546.989990


In [131]:
#計算ma
def moving_average(data,periods):
    return data['Close'].rolling(periods).mean()

In [132]:
df_copy = df.copy()
df_copy['min'] = df_copy['Low'].rolling(9).min()
df_copy['max'] = df_copy['High'].rolling(9).min()
df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) /(df_copy['max'] - df_copy['min'])
df_copy = df_copy.dropna()
K_list = [50]
for index,rsv in enumerate(list(df_copy['RSV'])):
    K_yesterday = K_list[index]
    K_today = 2/3 * K_yesterday + 1/3 *rsv
    K_list.append(K_today)
df_copy['K'] = K_list[1:]
D_list = [50]
for index,K in enumerate(list(df_copy['K'])):
    D_yesterday = D_list[index]
    D_today = 2/3 * D_yesterday + 1/3 * K
    D_list.append(D_today)
df_copy['D'] = D_list[1:]

In [133]:
df_copy.head()

,Open,High,Low,Close,Adj Close,Volume,min,max,RSV,K,D
Date,,,,,,,,,,,
2020-03-24,254.330002,260.529999,250.800003,260.209991,257.898468,1427600,191.25,219.240005,2.463736,34.154579,44.718193
2020-03-25,260.670013,272.339996,251.949997,262.070007,259.741943,1211300,191.25,219.240005,2.530189,23.613116,37.683167
2020-03-26,262.890015,277.140015,261.609985,275.899994,273.449097,1018600,191.25,219.240005,3.024294,16.750175,30.705503
2020-03-27,262.209991,263.709991,251.809998,252.619995,250.375885,1518300,191.25,219.240005,2.192568,11.897639,24.436215
2020-03-30,263.570007,270.130005,260.690002,268.029999,265.649017,1142400,191.25,219.240005,2.743122,8.846134,19.239521


In [134]:
#計算KD值
'''
step1 計算RSV:(今日的收盤價 - 最近9天的最低價)/(最近9天的最高價-最近9天的最低價)
step2 計算K值: K = 2/3 * (昨天的K值) + 1/3 * (今日RSV)
step2 計算D值: K = 2/3 * (昨天的D值) + 1/3 * (今日的K值)
'''
def KD(data):
    #step1 計算RSV
    df_copy = data.copy()
    df_copy['min'] = df_copy['Low'].rolling(9).min()
    df_copy['max'] = df_copy['High'].rolling(9).min()
    df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) /(df_copy['max'] - df_copy['min'])
    #step2 計算K值
    df_copy = df_copy.dropna()
    K_list = [50]
    for index,rsv in enumerate(list(df_copy['RSV'])):
        K_yesterday = K_list[index]
        K_today = 2/3 * K_yesterday + 1/3 *rsv
        K_list.append(K_today)
    df_copy['K'] = K_list[1:]
    #step3 計算D值
    D_list = [50]
    for index,K in enumerate(list(df_copy['K'])):
        D_yesterday = D_list[index]
        D_today = 2/3 * D_yesterday + 1/3 * K
        D_list.append(D_today)
    df_copy['D'] = D_list[1:]
    use_df = pd.merge(data,df_copy[['K','D']],left_index=True,right_index=True,how='left')
    return use_df

In [135]:
KD(df).tail(10)

,Open,High,Low,Close,Adj Close,Volume,K,D
Date,,,,,,,,
2021-02-26,572.340027,573.489990,560.700012,567.090027,567.090027,946500,1.291354,2.271222
2021-03-01,572.369995,584.359985,568.859985,583.549988,583.549988,679500,1.436704,1.993049
2021-03-02,579.030029,579.869995,563.000000,563.140015,563.140015,696100,1.041679,1.675926
2021-03-03,564.590027,565.950012,541.200012,543.599976,543.599976,1006500,0.726775,1.359542
2021-03-04,537.849976,541.330017,502.089996,510.500000,510.500000,1701600,0.555958,1.091681
2021-03-05,527.969971,530.849976,508.329987,527.039978,527.039978,1369400,0.659813,0.947725
2021-03-08,526.440002,529.169983,501.109985,502.190002,502.190002,1174900,0.452705,0.782718
2021-03-09,527.640015,543.210022,524.950012,539.520020,539.520020,1239700,0.758088,0.774508
2021-03-10,540.080017,541.820007,521.109985,523.330017,523.330017,876500,0.769351,0.772789
